# Малютин Павел, ИТПЗ РАН
 

In [1]:
import pandas as pd
import math 
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from numba import jit, njit
from sklearn.linear_model import LinearRegression
import random

# b-value calculating

In [2]:
'''чтение файла из директории проги
и устранение выбросов значений магнитуды округлением до 1 знака пз.'''
my_cat = pd.read_csv('./catalogues/DOUBLE_PREC_DD (2).csv', sep = ';') 
my_cat = my_cat.loc[my_cat.latitude > 38.7751667]
my_cat = my_cat.loc[my_cat.latitude < 38.8081667]
my_cat = my_cat.loc[my_cat.longitude > -122.778]
my_cat = my_cat.loc[my_cat.longitude < -122.733]
my_cat = my_cat.reset_index(drop = True)
my_cat.mag = round(my_cat.mag, 1)  
my_cat.SECOND = my_cat.SECOND.astype('int')
my_cat = my_cat.loc[my_cat['mag'] > 1.1]
my_cat = my_cat.reset_index(drop = True)
global N_tot 
N_tot = my_cat.shape
my_cat

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,latitude,longitude,depth,EH1,EH2,AZ,EV,mag,ID
0,1984,1,5,22,28,2,38.78798,-122.77681,1.663,0.016,0.015,82,0.021,2.6,1109586
1,1984,1,10,6,40,46,38.78415,-122.75795,1.134,0.018,0.011,158,0.023,1.4,1334
2,1984,1,10,16,7,36,38.78980,-122.77341,1.402,0.025,0.010,77,0.027,1.4,1109753
3,1984,1,13,16,54,19,38.78897,-122.77362,2.121,0.026,0.020,73,0.031,1.3,1109875
4,1984,1,14,3,52,25,38.78243,-122.76226,1.150,0.016,0.014,173,0.029,2.1,1109891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9555,2011,12,26,18,47,33,38.79207,-122.75521,2.075,0.010,0.007,50,0.009,1.7,71700500
9556,2011,12,29,7,48,17,38.77648,-122.75740,1.148,0.010,0.006,170,0.008,2.1,71702075
9557,2011,12,30,5,1,42,38.78675,-122.74343,2.123,0.006,0.005,12,0.004,1.5,71702715
9558,2011,12,31,2,14,36,38.80543,-122.74067,2.475,0.007,0.005,33,0.008,1.4,71703210


In [3]:
n_ = np.load('n_.npy')
resultate = pd.read_csv('./results/nearest_dist_n_index.csv')
''''''
index_o_cn = resultate.Index_o_CN
closest_neigh = resultate.Closest_neigh
less_than_temp = []
for i in range(N_tot[0]-1,-1,-1):
    if closest_neigh[i] < 1.8:
        less_than_temp.append(i)

In [4]:
resultate

,Unnamed: 0,Closest_neigh,Index_o_CN
0,0,2.5,33
1,1,2.0,8
2,2,2.7,0
3,3,3.1,0
4,4,3.1,81
...,...,...,...
9555,9555,2.3,8532
9556,9556,3.7,9557
9557,9557,2.7,9516
9558,9558,3.1,9507


In [5]:
my_cat = my_cat.drop(my_cat.index[less_than_temp]) 
my_cat = my_cat.reset_index(drop = True)

In [6]:

my_cat

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,latitude,longitude,depth,EH1,EH2,AZ,EV,mag,ID
0,1984,1,5,22,28,2,38.78798,-122.77681,1.663,0.016,0.015,82,0.021,2.6,1109586
1,1984,1,10,6,40,46,38.78415,-122.75795,1.134,0.018,0.011,158,0.023,1.4,1334
2,1984,1,10,16,7,36,38.78980,-122.77341,1.402,0.025,0.010,77,0.027,1.4,1109753
3,1984,1,13,16,54,19,38.78897,-122.77362,2.121,0.026,0.020,73,0.031,1.3,1109875
4,1984,1,14,3,52,25,38.78243,-122.76226,1.150,0.016,0.014,173,0.029,2.1,1109891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6989,2011,12,26,18,47,33,38.79207,-122.75521,2.075,0.010,0.007,50,0.009,1.7,71700500
6990,2011,12,29,7,48,17,38.77648,-122.75740,1.148,0.010,0.006,170,0.008,2.1,71702075
6991,2011,12,30,5,1,42,38.78675,-122.74343,2.123,0.006,0.005,12,0.004,1.5,71702715
6992,2011,12,31,2,14,36,38.80543,-122.74067,2.475,0.007,0.005,33,0.008,1.4,71703210


In [7]:
TIME_LIST = my_cat['YEAR'].astype(str)+ " " + my_cat['MONTH'].astype(str)+" " + my_cat['DAY'].astype(str)+ " " + my_cat['HOUR'].astype(str)+ " " + my_cat['MINUTE'].astype(str)+ " " + my_cat['SECOND'].astype(str)

In [8]:
random.shuffle(TIME_LIST)
my_cat['TIME'] = TIME_LIST

In [9]:
my_cat

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,latitude,longitude,depth,EH1,EH2,AZ,EV,mag,ID,TIME
0,1984,1,5,22,28,2,38.78798,-122.77681,1.663,0.016,0.015,82,0.021,2.6,1109586,2011 11 11 13 23 49
1,1984,1,10,6,40,46,38.78415,-122.75795,1.134,0.018,0.011,158,0.023,1.4,1334,1986 1 5 0 47 11
2,1984,1,10,16,7,36,38.78980,-122.77341,1.402,0.025,0.010,77,0.027,1.4,1109753,1996 9 2 3 16 1
3,1984,1,13,16,54,19,38.78897,-122.77362,2.121,0.026,0.020,73,0.031,1.3,1109875,2004 8 4 11 19 0
4,1984,1,14,3,52,25,38.78243,-122.76226,1.150,0.016,0.014,173,0.029,2.1,1109891,1998 1 9 21 51 17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6989,2011,12,26,18,47,33,38.79207,-122.75521,2.075,0.010,0.007,50,0.009,1.7,71700500,1990 2 28 22 43 57
6990,2011,12,29,7,48,17,38.77648,-122.75740,1.148,0.010,0.006,170,0.008,2.1,71702075,2001 4 19 21 36 9
6991,2011,12,30,5,1,42,38.78675,-122.74343,2.123,0.006,0.005,12,0.004,1.5,71702715,2003 1 8 4 29 24
6992,2011,12,31,2,14,36,38.80543,-122.74067,2.475,0.007,0.005,33,0.008,1.4,71703210,1994 1 2 22 43 20


In [10]:
my_cat[['YEAR','MONTH','DAY','HOUR','MINUTE','SECOND']] = my_cat.TIME.str.split(expand=True)
my_cat

,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,latitude,longitude,depth,EH1,EH2,AZ,EV,mag,ID,TIME
0,2011,11,11,13,23,49,38.78798,-122.77681,1.663,0.016,0.015,82,0.021,2.6,1109586,2011 11 11 13 23 49
1,1986,1,5,0,47,11,38.78415,-122.75795,1.134,0.018,0.011,158,0.023,1.4,1334,1986 1 5 0 47 11
2,1996,9,2,3,16,1,38.78980,-122.77341,1.402,0.025,0.010,77,0.027,1.4,1109753,1996 9 2 3 16 1
3,2004,8,4,11,19,0,38.78897,-122.77362,2.121,0.026,0.020,73,0.031,1.3,1109875,2004 8 4 11 19 0
4,1998,1,9,21,51,17,38.78243,-122.76226,1.150,0.016,0.014,173,0.029,2.1,1109891,1998 1 9 21 51 17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6989,1990,2,28,22,43,57,38.79207,-122.75521,2.075,0.010,0.007,50,0.009,1.7,71700500,1990 2 28 22 43 57
6990,2001,4,19,21,36,9,38.77648,-122.75740,1.148,0.010,0.006,170,0.008,2.1,71702075,2001 4 19 21 36 9
6991,2003,1,8,4,29,24,38.78675,-122.74343,2.123,0.006,0.005,12,0.004,1.5,71702715,2003 1 8 4 29 24
6992,1994,1,2,22,43,20,38.80543,-122.74067,2.475,0.007,0.005,33,0.008,1.4,71703210,1994 1 2 22 43 20


In [11]:
b_value = 0.855
d_f = 2.08
M_0 = 1.1
 
N_tot = my_cat.shape

In [12]:
T_ = [[0] * N_tot[0] for i in range(N_tot[0])]

In [31]:
n_ = [[float('inf')] * N_tot[0] for i in range(N_tot[0])]

In [13]:
import datetime
year = my_cat.YEAR.to_numpy().astype(int)
month = my_cat.MONTH.to_numpy().astype(int)
day = my_cat.DAY.to_numpy().astype(int)
hour = my_cat.HOUR.to_numpy().astype(int)
minute = my_cat.MINUTE.to_numpy().astype(int)
second = my_cat.SECOND.to_numpy().astype(int)

In [14]:
@jit
def time_in_seconds (i, j):
    t_i = datetime.datetime(year[i], month[i], day[i], hour[i], minute[i], second[i])
    t_j = datetime.datetime(year[j], month[j], day[j], hour[j], minute[j], second[j])
    t_ij = (t_i - t_j).total_seconds()
    if t_ij == 0:
        return -0.1
    return -t_ij

In [15]:
for i in tqdm(range(N_tot[0]-1,-1,-1)):
    for j in range(i,-1,-1):
        T_[i][j] = -time_in_seconds(i,j)
        #T_[j][i] = T_[i][j]

  0%|                                                 | 0/6994 [00:00<?, ?it/s]C:\Users\admin\AppData\Local\Temp\ipykernel_3312\1257614798.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "time_in_seconds" failed type inference due to: Unknown attribute 'datetime' of type Module(<module 'datetime' from 'C:\\Users\\admin\\anaconda3\\lib\\datetime.py'>)

File "..\..\..\AppData\Local\Temp\ipykernel_3312\1257614798.py", line 3:
<source missing, REPL/exec in use?>

During: typing of get attribute at C:\Users\admin\AppData\Local\Temp\ipykernel_3312\1257614798.py (3)

File "..\..\..\AppData\Local\Temp\ipykernel_3312\1257614798.py", line 3:
<source missing, REPL/exec in use?>

  @jit
C:\Users\admin\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "time_in_seconds" was compiled in object mode without forceobj=True.

File "..\..\..\AppData\Local\Temp\ipykernel_3312\1257614798.py", line 1:
<source mis

In [16]:
global Pi2, RAD, FLAT
Pi2 = math.pi/2 # тупо пи-на-два
RAD = 0.01745329 # DGR -> RAD
FLAT = 0.993231 # поправка на эллипсоидноть
T_[1][2]

0

In [17]:
def eq_dist (fi1, la1, fi2, la2, d1, d2): # fi - широта (latitude), la - долгота (longtitude)
    '''Функция позволяет вычислять расстояение между двумя точками земли с учетом сплюсности и глубины'''    
    #if ((fi1-fi2)==0 and (la1-la2)==0): #проверка на расстояние между одним и тем же землетрясением
    #    return (0.0)  
    col1 = Pi2 - math.atan(FLAT * math.tan(fi1 * RAD))
    col2 = Pi2 - math.atan(FLAT * math.tan(fi2 * RAD))
    
    dlon =  la2 * RAD - la1 * RAD
    
    cosdel = math.sin(col1) * math.sin (col2) * math.cos(dlon) + math.cos(col1) * math.cos(col2)
    delr = math.acos(cosdel)
    colat = Pi2 - (fi1 * RAD + fi2 * RAD) / 2.0
    distan_eq =  math.sqrt((delr * 6371.227 * (1.0 + 0.00337853 * (1./3. -((math.cos(colat))*math.cos(colat)))))**2 + (d2-d1)**2)
    if distan_eq == 0:
        distan_eq = 0.01
    return distan_eq

In [18]:
latitude = my_cat.latitude.to_numpy()
longitude = my_cat.longitude.to_numpy()
depth = my_cat.depth.to_numpy()

In [19]:
array_of_dist = [[0] * N_tot[0] for i in range(N_tot[0])]

for i in tqdm(range((N_tot[0]))):
    for j in range(i+1, N_tot[0]):
        #array_of_dist.append(eq_dist(latitude[k], longitude[k], latitude[j], longitude[j], depth[k], depth[j]))
        array_of_dist[i][j] = eq_dist(latitude[i], longitude[i], latitude[j], longitude[j], depth[i], depth[j])
        #array_of_dist[j][i] = array_of_dist[i][j]

100%|██████████████████████████████████████| 6994/6994 [03:25<00:00, 34.10it/s]


In [36]:
for i in tqdm(range(N_tot[0])):
    for j in range(i,N_tot[0]): #ВНИМАНИЕ - ХЗ ПОЧЕМУ ОТ И БЕРУ. ХОЧУ ПРОВЕРИТЬ
        if i == j:
            continue 
        n_[j][i] = abs(T_[j][i])*(array_of_dist[i][j]**d_f)*(10**((-b_value)*my_cat.mag[i]))
        #n_[j][i] = n_[i][j]

  5%|██                                     | 364/6994 [00:45<13:57,  7.92it/s]


KeyboardInterrupt: 

In [ ]:
N = [0] * N_tot[0]
D = [0] * N_tot[0]
for i in range(len(n_)):
    D[i] = round(math.log10((min(n_[i][:]))),1)
    N[i] = n_[:][i].index(min(n_[i][:]))
resultate = pd.DataFrame({'Closest_neigh':D, 'Index_o_CN':N})

In [ ]:
Dist_to_Nearest = resultate.Closest_neigh.value_counts(normalize=True)
from pathlib import Path  
filepath = Path('./results/nearest_dist_rand.csv')
Dist_to_Nearest.to_csv(filepath)